# Sesión práctica: Clima y salud

### Ana Casanueva Vicente y Rodrigo García Manzanas, 3 de diciembre de 2024

En esta práctica se muestra una introducción a la estimación de **estrés térmico**, como combinación de **temperatura y humedad**, usando datos de observaciones y de modelos climáticos.
Además, como se ha visto en la sesión teórica, debido a que los modelos climáticos presentan ciertos sesgos sistemáticos, se han de aplicar **técnicas de ajuste o corrección de sesgos** antes de utilizarlos en  estudios de impactos, especialmente en aquellos que se relacionan con la superación de umbrales de ciertos índices, como en el caso de estŕes térmico.

## 1. Preparación del entorno y carga de paquetes

Para la lectura, manipulación y visualización de los datos climáticos se van a usar paquetes de `R` desarrollados por el Grupo de Meteorología de Santander (Universidad de Cantabria y CSIC), que forman el conjunto de paquetes de `climate4R` (https://github.com/SantanderMetGroup/climate4R, [Iturbide et al. 2019](https://doi.org/10.1016/j.envsoft.2018.09.009)). Estos paquetes permiten la lectura (remota o local, `loadeR`), transformaciones básicas como interpolaciones, generación de subconjuntos, etc. (`transformeR`), cálculo de índices climáticos (`climate4R.indices`), regionalización estadística (_downscaling_) y corección de sesgos (`downscaleR`) y representaciones gráficas (`visualizeR`). Todos estos paquetes ya están instalados en esta plataforma, por lo que directamente se cargarán al espacio de trabajo mediante la función `library`.

In [ ]:
rm(list=ls())
graphics.off()
library(loadeR) # para leer datos (función loadGridData)
library(visualizeR) # para generar figuras (función spatialPlot)
library(downscaleR) # para bias correction (función biasCorrection)
library(transformeR) # para transformaciones, p.ej. subset (subsetDimension, subsetGrid), interpolar (interpGrid), etc.
library(climate4R.indices) # para calcular índices, p.ej. indexGrid

Para el cálculo de índices de estrés térmico se usará el paquete `HeatStress` (https://github.com/anacv/HeatStress), que se tiene que instalar previamente desde `GitHub` con el paquete `remotes`. La instalación solo se es necesaria la primera vez que se usan. Una vez instalamos y cargamos `HeatStress`, con la función `indexShow()` podemos ver los índices implementados, así como las variables que requieren como argumentos de entrada.

In [ ]:
# install.packages("remotes")
# install.packages("assertthat")

In [ ]:
# library(remotes)
# remotes::install_github("anacv/HeatStress")

In [ ]:
library(assertthat)
library(HeatStress)
indexShow()
?swbgt

## 2. Lectura de datos de (pseudo)observaciones

En cualquier estudio climático el punto de partida son los datos de observaciones, que nos permiten caracterizar las condiciones climáticas de un lugar y estudiar tendencias, así como evaluar modelos (i.e. cuantificar cuán buenos son), entre otros. En esta práctica vamos a trabajar con datos de modelos climáticos y observaciones sobre un dominio rectangular que cubre Perú. Los datos de observaciones _in situ_ son a menudo poco accesibles y se tienen en localizaciones puntuales. Por este motivo a menudo se usan datos _en rejilla_ que cubren todo el globo o una región determinada. Estas rejillas se elaboran con métodos de interpolación a partir de las observaciones _in situ_ o, como en el caso de los reanálisis, son el resultado de un modelo climático que se alimenta de forma continua con observaciones _in situ_. Una ventaja de este tipo de datos es que proporciona un amplio conjunto de variables meteorológicas, como en nuestro caso, en que usaremos temperatura y humedad. 
Los datos que usaremos como observaciones son datos del reanálisis de última generación [ERA5](https://cds.climate.copernicus.eu/datasets/derived-era5-pressure-levels-daily-statistics?tab=overview), desarrollado por el Centro Europeo ECMWF dentro el programa Copernicus, que proporciona datos globales y se actualiza casi a tiempo real.

Para facilitar la lectura, para esta práctica se ha seleccionado una porción de ERA5 que cubre el dominio de estudio, y que leemos con el comando `load` usando una ruta local donde se encuentran los datos. Los datos están en una rejilla regular con resolución espacial de 0.25ºx0.25º, que equivale aproximadamente a 25kmx25km.
A continuación se lee la **temperatura máxima diaria** (`tmax.obs`) y la **humedad relativa media diaria** (`relhum.obs`). Hay distintas funciones para explorar los datos, una de ellas es `str` que nos muestra su estructura.

In [ ]:
tmax.obs <- get(load("../shared/data/curso_UNASAM/tasmax.era5.rda", verbose=TRUE))
relhum.obs <- get(load("../shared/data/curso_UNASAM/hurs.era5.rda", verbose=TRUE))
rm(hurs.era5, tasmax.era5)

In [ ]:
str(tmax.obs)

Podemos observar que los datos vienen en una lista que contiene información de la variable (incluyendo unidades), los datos propiamente dichos, las coordenadas espaciales de la rejilla de los datos e información de las fechas. Notar que los datos forman un _array_ tridimensional, cuyas dimensiones se refieren a los tiempos, latitudes y longitudes. 
A continuación comprobamos cuál es el rango de fechas, y usamos `subsetGrid` para quedarnos con un periodo de 20 años en las dos variables (temperatura máxima y humedad relativa media diarias).

In [ ]:
range(tmax.obs$Dates)

In [ ]:
tmax.obs <- subsetGrid(tmax.obs, years = 1986:2005)
relhum.obs <- subsetGrid(relhum.obs, years = 1986:2005)

Por comodidad, vamos a pasar los datos de temperatura de Kelvin (K) a grados centígrados, usando la función `gridArithmetics` para restar por el valor adecuado.

In [ ]:
tmax.obs <- gridArithmetics(tmax.obs, 273.15, operator="-")

Para ver cuál es el patrón espacial de la temperatura máxima diaria en Perú, haremos un promedio temporal sobre los 20 años (lo denominamos **climatología**) con la función `climatology`y la representamos con `spatialPlot`. En la figura se observa claramente el efecto de la cordillera, de la costa y de la selva en la temperatura máxima diaria.

In [ ]:
spatialPlot(climatology(tmax.obs), backdrop.theme = "countries", rev.colors = TRUE,  at= seq(5,35),
            main= "Temperatura maxima diaria observada (1986-2005)")

Análogamente para la humedad relativa:

In [ ]:
spatialPlot(climatology(relhum.obs), backdrop.theme = "countries",  color.theme = "GnBu", at=seq(20,100,10),
            main= "Humedad media diaria observada (1986-2005)")

En el caso de la humedad, hemos cambiado la barra de color para que sea acorde e intuitiva con la variable que pintamos. Para ello, se puede cambiar el argumento `color.theme` que admite las barras de color de [ColorBrewer](https://r-graph-gallery.com/38-rcolorbrewers-palettes.html).

## 3. Cálculo de índices de estrés térmico de las observaciones

Una vez tenemos listos los datos de temperatura y humedad relativa, calcularemos uno de los índices de estrés térmico disponibles en el paquete `HeatStress`, concretamente el _simplified wet bulb globe temperature_, que depende de la temperatura y de la humedad relativa y, como se ha visto en la sesión teórica, se utiliza habitualmente para estimar el estrés térmico en el población trabajadora. Este índice es el más utilizado en las normas ISO que proporcionan recomendaciones acerca de la exposición al calor de la población trabajadora. La función para calcular este índice es `swbgt`, cuyos argumentos de entrada son vectores, por lo que recorremos en dos bucles todos los puntos de la rejilla de datos para las dos variables.

In [ ]:
wbgt.obs <- tmax.obs
for(i in 1:dim(tmax.obs$Data)[2]){
	for(j in 1:dim(tmax.obs$Data)[3]){
		wbgt.obs$Data[,i,j]<- swbgt(tmax.obs$Data[,i,j], relhum.obs$Data[,i,j])	
	}
}

Hacemos una representación de la climatología (promedio temporal) de la temperatura máxima junto con la del índice de estrés térmico. Notar que, a pesar de que ambas se expresan en grados centígrados, el índice de estrés térmico es sistemáticamente menor, por el efecto de la humedad relativa. 

In [ ]:
options(repr.plot.width=10, repr.plot.height=6) # cambia el tamaño de la imagen
spatialPlot(makeMultiGrid(climatology(tmax.obs), climatology(wbgt.obs)), backdrop.theme = "countries", rev.colors = TRUE, 
            as.table=TRUE, names.attr=c("Temperatura máxima", "Estrés térmico"), layout=c(2,1),
            main= "Temperatura y estrés térmico observado (1986-2005)")

Además de analizar la climatología, podemos estudiar la frecuencia de ciertos eventos, por ejemplo, el número de días en que se superan ciertos umbrales cada año. En concreto usamos los umbrales de 26ºC y 28ºC que se relacionan con situaciones de riesgo para personas no aclimatadas y aclimatadas, respectivamente, que realizan una actividad física moderada. Para calcular el número de días en que se supera un umbral usamos la función `indexGrid`. Al indicar que la resolución temporal sea anual (`year`) obtenemos una serie con el número de días en que se supera cada umbral para cada uno de los 20 años (1986-2005).

In [ ]:
wbgt26.obs <- indexGrid(tx = wbgt.obs, time.resolution = "year", index.code = "TXth", th=26)
wbgt28.obs <- indexGrid(tx = wbgt.obs, time.resolution = "year", index.code = "TXth", th=28)

Analizamos la evolución temporal en el punto de la rejilla más cercano a la ciudad de Pucallpa, en la Amazonia peruana (latitud -8.379 y longitud -74.554), que se encuentra en la zona más afectada por estrés térmico. Para ello, en primer lugar, filtramos los valores para este punto con `subsetDimension` y representamos los resultados con `temporalPlot`.

In [ ]:
wbgt26.1grid <- subsetDimension(subsetDimension(wbgt26.obs, dimension = "lat", indices = 44), dimension = "lon", indices = 31)
wbgt28.1grid <- subsetDimension(subsetDimension(wbgt28.obs, dimension = "lat", indices = 44), dimension = "lon", indices = 31)

In [ ]:
temporalPlot(wbgt26.1grid, wbgt28.1grid, cols=c("dodgerblue","coral1"), lwd = 3, 
	xyplot.custom = list(main = "Evolución temporal del estrés térmico en Pucallpa",  ylab= "Nº de días", scales=list(x=list(cex=1), y=list(cex=1)), 
                         key = list(space = "right", points = list(pch= 15, col = c("dodgerblue","coral1"), cex = 1.8), text = list(c("WBGT>26","WBGT>28"), cex = 1.5))))

Representamos la climatología (valor promedio de los 20 años) del número de días al año en que se exceden los umbrales para todos los puntos, usando de nuevo `spatialPlot`. Se observa que en amplias regiones se supera el umbral de 26ºC casi todos los días del año y el umbral de 28ºC al menos la mitad de los días del año.

In [ ]:
spatialPlot(makeMultiGrid(climatology(wbgt26.obs),climatology(wbgt28.obs)), backdrop.theme = "countries",  
            as.table=TRUE, names.attr=c("No aclimatados (WBGT > 26)", "Aclimatados (WBGT > 28)"), layout=c(2,1), 
            rev.colors = TRUE, main= "Días con estrés térmico najo actividad moderada (1986-2005)")

## 4. Datos de modelos climáticos

En esta sesión vamos a utilizar un **Modelo Regional del Clima**, concretamente _REMO2015_ anidado a un modelo global del clima (_MPI-ESM_LR_) desarrollado en una rejilla de resolución espacial 0.22ºx0.22º (similar a la de las observaciones). Dado que este modelo tiene una malla rotada, se ha interpolado previamente y de forma conservativa a la rejilla de la observación. Concretamente se van a utilizar dos tipos de simulaciones:  _histórica_ (1986-2005) y de _futuro_ para un escenario de emisiones fuertes de cambio climático. La segunda cubre desde 2006 a 2100, pero consideraremos un periodo de 20 años al final del siglo para aligerar los cálculos (2081-2100). Al igual que con las observaciones, consideraremos temperatura máxima y humedad relativa media diarias, aunque en esta sección nos centraremos en la temperatura máxima. 
Todos los datos del modelo se han desarrollado en el marco de la iniciativa **CORDEX** (https://cordex.org/) que coordina los esfuerzos internacionales en la generación de proyecciones climáticas a escala regional. 

### 4.1. Lectura de los datos

Dada la estructura más compleja de estos datos (distribuidos en numerosos archivos), utilizamos la función `loadGridData`, que permite leer la variable deseada y seleccionar en la lectura un periodo de tiempo y un dominio espacial.

In [ ]:
dataset.sim.hist <- "../shared/data/ncml/ESGF/interp025/CORDEX/output/SAM-22/GERICS/MPI-M-MPI-ESM-LR/historical/r1i1p1/REMO2015/v1/day/CORDEX_output_SAM-22_GERICS_MPI-M-MPI-ESM-LR_historical_r1i1p1_REMO2015_v1_day.ncml"
dataset.sim.fut <-"../shared/data/ncml/ESGF/interp025/CORDEX/output/SAM-22/GERICS/MPI-M-MPI-ESM-LR/rcp85/r1i1p1/REMO2015/v1/day/CORDEX_output_SAM-22_GERICS_MPI-M-MPI-ESM-LR_rcp85_r1i1p1_REMO2015_v1_day.ncml"

In [ ]:
tmax.sim.hist <- loadGridData(dataset.sim.hist, var="tasmax", years= 1986:2005, lonLim = c(-82, -68), latLim = c(1, -19))
str(tmax.sim.hist)

Comprobamos como anteriormente que tenemos el periodo deseado, y que las observaciones y el modelo tienen la misma rejilla espacial. De lo contrario habría que hacer una intepolación espacial.

In [ ]:
range(tmax.sim.hist$Dates)

In [ ]:
str(tmax.sim.hist$xyCoords)
str(tmax.obs$xyCoords)

Análogamente, leemos los datos del periodo futuro de finales de siglo.

In [ ]:
tmax.sim.fut <- loadGridData(dataset.sim.fut, var="tasmax", years= 2081:2100, lonLim = c(-82, -68), latLim = c(1, -19))
str(tmax.sim.fut)

### 4.2. Preparación de los datos

Por comodidad, hacemos un cambio de unidades para temperatura, de Kelvin a grados centígrados. Lo hacemos para los datos de la simulación histórica (periodo 1986-2005) y de la simulación de futuro (2081-2100).

In [ ]:
tmax.sim.hist <- gridArithmetics(tmax.sim.hist, 273.15, operator="-")
tmax.sim.fut <- gridArithmetics(tmax.sim.fut, 273.15, operator="-")

Al representar la climatología (valor promedio) de la temperatura máxima diaria en los dos periodos, comprobamos que con el paso del tiempo, el modelo muestra un calentamiento en toda la región.

In [ ]:
spatialPlot(makeMultiGrid(climatology(tmax.sim.hist), climatology(tmax.sim.fut), skip.temporal.check=TRUE), backdrop.theme = "countries",
            rev.colors = TRUE, at= seq(5,35), set.max=35,
            as.table=TRUE, names.attr=c("Simulación histórica (1981-2005)", "Simulación futura (2081-2100)"), layout=c(2,1),
            main= "Temperatura maxima diaria simulada")

### 4.2. Sesgos en las simulaciones de modelos

Como se indicó anteriormente, las observaciones se usan a menudo para la **evaluación de modelos**. En esta sección mostramos distintos aspectos de la evaluación del modelo considerado. 
En primero lugar, podemos comparar las climatologías (valores promedio) del modelo y la observación en el periodo 1986-2005. En la figura se observa que el modelo muestra temperaturas más altas que la observación en la costa y en la zona de la selva y más bajas en algunos puntos de la cordillera.

In [ ]:
spatialPlot(makeMultiGrid(climatology(tmax.obs), climatology(tmax.sim.hist)), backdrop.theme = "countries", 
            rev.colors = TRUE, at= seq(5,35), 
            as.table=TRUE, names.attr=c("Observación", "Modelo"), layout=c(2,1),
            main= "Temperatura maxima diaria (1981-2005)")

Cuantificamos ese error en la media temporal calculando el sesgo (_modelo - observación_) usando la función `gridArithmetics`.

In [ ]:
bias.tmax.raw <- gridArithmetics(climatology(tmax.sim.hist), climatology(tmax.obs),operator="-")
options(repr.plot.width=10, repr.plot.height=6) # cambia el tamaño de la imagen
spatialPlot(bias.tmax.raw, backdrop.theme = "countries", 
            rev.colors = TRUE, at= seq(-10,10), 
            main= "Sesgo medio en la temperatura maxima diaria (1981-2005)")

Podemos analizar en detalle la distribución completa de los datos en un punto de la rejilla concreto. Aquí consideramos el punto más cercano a la ciudad de Huaraz (latitud -9.528, longitud -77.528). Representamos en la siguiente figura el histograma de la temperatura observada en gris y del modelo en rosa. Se observa que en este punto en concreto, el modelo tiene un sesgo positivo de aproximadamente 2ºC en la media, pero además la distribución del modelo tiene más variabilidad que la observación. 

In [ ]:
hist(tmax.obs$Data[,39,19], col=rgb(0,0,0,,alpha=0.2), main="Distribución de datos en un punto cerca de Huaraz", 
     xlab="Temperatura máxima", xlim=c(0,25))
hist(tmax.sim.hist$Data[,39,19],  col=rgb(1,0,0,,alpha=0.3), add=TRUE)
legend("topleft", legend=c("OBS","SIM (HIST)"), col=c(rgb(0,0,0,,alpha=0.2),rgb(1,0,0,,alpha=0.3)), 
       pch=16, pt.cex=2.5, horiz=FALSE, bty="n",cex =1.5)

Estos son algunos ejemplos de evaluaciones sencillas que podemos llevar a cabo. A la vista de los errores cometidos, no es conveniente utilizar los modelos directamente para estudios de impacto. Por lo tanto, llevaremos a cabo la **corrección o ajuste de sesgos**.

## 5. Corrección de sesgos en la simulación histórica

Como se ha visto en la sesión de teoría, hay una variedad enorme de métodos que permiten corregir distintas partes de la distribución. En este ejemplo vamos a probar dos métodos y ver el efecto que tienen en distintos aspectos: _scaling_ y _empirical quantile mapping (eqm)_. Dado que la temperatura máxima es una variable que sigue una distribución gaussiana, en el primer caso se realizará una correccióna aditiva (desplazamiento de la distribución de los datos). Para entrenar y aplicar las corecciones de los dos métodos utilizaremos la función `biasCorrection`.

In [ ]:
tmax.scaling.hist <- biasCorrection(y=tmax.obs, x=tmax.sim.hist, newdata= tmax.sim.hist, precipitation = FALSE, 
                             method="scaling", scaling.type="additive" ) # corrección solo de la media (aditiva para temperatura)
tmax.eqm.hist <- biasCorrection(y=tmax.obs, x=tmax.sim.hist, newdata= tmax.sim.hist, precipitation = FALSE, 
                         method="eqm", extrapolation = "constant", n.quantiles=99 )  # corrección de 99 percentiles

### 5.1. Efecto de la corrección en la media

A continuación evaluamos el efecto que tienen las distintas correcciones en diversos aspectos, en primer lugar, en la media de la distribución de temperatura. Para ello, calculamos el sesgo como anteriormente como _modelo - observación_ y representamos el sesgo de los datos sin corregir (_raw_) y de los datos corregidos mediante los dos métodos (_scaling_ y _eqm_).

In [ ]:
bias.tmax.scaling <- gridArithmetics(climatology(tmax.scaling.hist), climatology(tmax.obs), operator="-")
bias.tmax.eqm <- gridArithmetics(climatology(tmax.eqm.hist), climatology(tmax.obs), operator="-")

In [ ]:
spatialPlot(makeMultiGrid(bias.tmax.raw, bias.tmax.scaling, bias.tmax.eqm), backdrop.theme = "countries", 
            rev.colors = TRUE,  at= seq(-10,10), main= "Sesgo en la temperatura máxima (promedio 1986-2005)", layout=c(3,1),
            as.table=TRUE, names.attr=c("RAW", "BC (scaling)", "BC (eqm)"))

Comprobamos que el sesgo medio se ha reducido prácticamente a cero al aplicar los dos tipos de correcciones.

### 5.2. Efecto de la corrección en la distribución

A pesar de que los dos métodos corrigen, por construcción, la media de la distribución, no tienen el mismo efecto en otras partes de la distribución. Recordemos que _scaling_ solo corrige la media (centra la distribución), mientras que _eqm_ corrige los 99 percentiles. Podemos ver el efecto sobre toda la distribución en los siguientes histogramas, donde se ha tomado de nuevo el punto de la rejilla más cercano a Huaraz.

In [ ]:
options(repr.plot.width=15, repr.plot.height=6) # cambia el tamaño de la imagen
par(mfrow=c(1,2))
hist(tmax.obs$Data[,39,19], col=rgb(0,0,0,,alpha=0.2), main="Distribución de datos en un punto cerca de Huaraz", 
     xlab="Temperatura máxima", xlim=c(0,25))
hist(tmax.scaling.hist$Data[,39,19],  col=rgb(1,0,0,,alpha=0.3), add=TRUE)
legend("topleft", legend=c("OBS","SCALING"), col=c(rgb(0,0,0,,alpha=0.2),rgb(1,0,0,,alpha=0.3)), pch=16,  pt.cex=2, horiz=FALSE, bty="n")

hist(tmax.obs$Data[,39,19], col=rgb(0,0,0,,alpha=0.2), main="Distribución de datos en un punto cerca de Huaraz", 
     xlab="Temperatura máxima", xlim=c(0,25))
hist(tmax.eqm.hist$Data[,39,19],  col=rgb(1,0,0,,alpha=0.3), add=TRUE)
legend("topleft", legend=c("OBS","EQM"), col=c(rgb(0,0,0,,alpha=0.2),rgb(1,0,0,,alpha=0.3)), pch=16, pt.cex=2, horiz=FALSE, bty="n")

También comprobamos el efecto de las correcciones con un gŕafico _q-q plot_:

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
x <- quantile(tmax.obs$Data[,39,19], seq(0.01,0.99,0.01), na.rm=T)
y <- quantile(tmax.sim.hist$Data[,39,19], seq(0.01,0.99,0.01), na.rm=T)
z <- quantile(tmax.scaling.hist$Data[,39,19], seq(0.01,0.99,0.01), na.rm=T)
w <- quantile(tmax.eqm.hist$Data[,39,19], seq(0.01,0.99,0.01), na.rm=T)

lims <- range(c(x,y,z))
plot(x,y, ylim=lims,xlim=lims, pch=16, xlab=NA, ylab=NA,asp=1, las=1)
par(new=T)
plot(x,z, ylim=lims,xlim=lims, pch=16, xlab=NA, ylab=NA,asp=1, las=1, col="dodgerblue")
par(new=T)
plot(x,w, ylim=lims,xlim=lims, pch=16, xlab="Observed percentiles", ylab="Simulated percentiles", col="coral1", cex.lab=1.5, asp=1, las=1)
lines(lims,lims, col="lightgrey", lty=2)
legend("bottomright", legend=c("RAW","SCALING","EQM"), col=c("black","dodgerblue","coral1"), pch=16, xpd=TRUE,horiz=FALSE, cex=1.5, bty="n")

### 5.3. Efecto de la corrección en la superación de umbrales

Dado que _eqm_ corrige los 99 percentiles, para una evaluación justa deberíamos evaluar algún aspecto que no haya ajustado explícitamente, por ejemplo, la frecuencia de superación de umbrales absolutos. Consideramos, por ejemplo, el número de días con tamperatura máxima por encima de 35ºC, calculados año a año con la función `indexGrid` para las observaciones, el modelo sin corregir (_raw_) y el modelo corregido por los dos métodos (_scaling_ y _eqm_):

In [ ]:
tx35.obs <- indexGrid(tx = tmax.obs, time.resolution = "year", index.code = "TXth", th=35)
tx35.raw.hist <- indexGrid(tx = tmax.sim.hist, time.resolution = "year", index.code = "TXth", th=35)
# biasCorrection devuelve fechas en un formato distinto que no funciona en indexGrid (ponemos fechas de RAW)
tmax.scaling.hist$Dates <- tmax.sim.hist$Dates
tmax.eqm.hist$Dates <- tmax.sim.hist$Dates
tx35.scaling.hist <- indexGrid(tx = tmax.scaling.hist, time.resolution = "year", index.code = "TXth", th=35)
tx35.eqm.hist <- indexGrid(tx = tmax.eqm.hist, time.resolution = "year", index.code = "TXth", th=35)

A continuación, calculamos el sesgo en la climatología (media interanual) del número de días con temperatura máxima por encima de 35ºC con `gridArithmetics`(_modelo - observación_) y se representa con `spatialPlot`.

In [ ]:
bias.tx35.raw <- gridArithmetics(climatology(tx35.raw.hist), climatology(tx35.obs), operator="-")
bias.tx35.scaling <- gridArithmetics(climatology(tx35.scaling.hist), climatology(tx35.obs), operator="-")
bias.tx35.eqm <- gridArithmetics(climatology(tx35.eqm.hist), climatology(tx35.obs), operator="-")

In [ ]:
options(repr.plot.width=10, repr.plot.height=6)
spatialPlot(makeMultiGrid(bias.tx35.raw, bias.tx35.scaling, bias.tx35.eqm), backdrop.theme = "countries", 
            rev.colors = TRUE,  at= seq(-70,70,5), main= "Sesgo en TX35", layout=c(3,1),
            as.table=TRUE, names.attr=c("RAW", "BC (scaling)", "BC (eqm)"))

Se comprueba que el modelo sin corregir (_raw_) tenía un sesgo muy alto y positivo de este indicador (sobrestimación de más de 40 días al año), _scaling_ reduce este sesgo pero todavía sobrestima con más de 20 días al año y _eqm_ presenta sesgos mucho más bajos de pocos días.

In [ ]:
rm(tx35.scaling.hist, bias.tx35.scaling, bias.tmax.raw, bias.tmax.scaling, bias.tmax.eqm, bias.tx35.eqm, bias.tx35.raw)

## 6. Corrección de sesgos la simulación de futuro

Después de realizar la evaluación considerando distintos aspectos, aplicamos el método _eqm_ a la simulación de futuro, para tener los valores de temperatura máxima diaria ajustada/corregida para el futuro. Para ello, seguimos usando la función `biasCorrection` pero en el argumento `newdata` introducimos los datos del modelo en la simulación de futuro. De esta manera, se calibra la corrección entre la observación y la simulación histórica, y se aplica sobre los datos de futuro.

In [ ]:
tmax.eqm.fut <- biasCorrection(y=tmax.obs, x=tmax.sim.hist, newdata= tmax.sim.fut, precipitation = FALSE, 
                         method="eqm", extrapolation = "constant", n.quantiles=99)  # corrección de 99 percentiles

### 6.1. Efecto de la corrección en la señal de cambio climático

Dependiendo del método de corrección que se use, se puede producir una modificación de la señal de cambio climático original del modelo. Llamamos **señal de cambio climático** (_climate change signal, ccs_) al cambio que proyecta el modelo en el futuro respecto a su histórico (_futuro - histórico_). Concretamente, _eqm_ por construcción puede dar lugar a estar modificaciones. Analizamos cuál es el efecto de la corrección con _eqm_ en la señal de cambio climático.

In [ ]:
ccs.tmax.raw <- gridArithmetics(climatology(tmax.sim.fut), climatology(tmax.sim.hist), operator="-")
ccs.tmax.eqm <- gridArithmetics(climatology(tmax.eqm.fut), climatology(tmax.eqm.hist), operator="-")

In [ ]:
spatialPlot(makeMultiGrid(ccs.tmax.raw, ccs.tmax.eqm), backdrop.theme = "countries", color.theme = "YlOrRd",
            main= "Señal de cambio en la temperatura máxima diaria (2081-2100 wrt 1986-2005)", 
            layout=c(2,1), as.table=TRUE, names.attr=c("RAW", "EQM"))

Ambas señales son positivas en todo el dominio (aumenta la temperatura máxima). Se observa que _eqm_ está atenuando la señal original del modelo en la media de la distribución.

Análogamente, analizamos el efecto en el número de días con temperatura máxima por encima de 35ºC.

In [ ]:
tx35.raw.fut <- indexGrid(tx = tmax.sim.fut, time.resolution = "year", index.code = "TXth", th=35)
tmax.eqm.fut$Dates <- tmax.sim.fut$Dates
tx35.eqm.fut <- indexGrid(tx = tmax.eqm.fut, time.resolution = "year", index.code = "TXth", th=35)

In [ ]:
ccs.tx35.raw <- gridArithmetics(climatology(tx35.raw.fut), climatology(tx35.raw.hist), operator="-")
ccs.tx35.eqm <- gridArithmetics(climatology(tx35.eqm.fut), climatology(tx35.eqm.hist), operator="-")

In [ ]:
spatialPlot(makeMultiGrid(ccs.tx35.raw, ccs.tx35.eqm), backdrop.theme = "countries", color.theme = "YlOrRd",
            main= "Señal de cambio en TX35 (2081-2100 wrt 1986-2005)", 
            layout=c(2,1), as.table=TRUE, names.attr=c("RAW", "EQM"))

También se observa que _eqm_ reduce la señal que proyecta el modelo, pasando de incrementos de más de 100 días a incrementos de alrededor de 50 días. Aun así, ambas señales dan un aumento en el número de días con temperatura máxima por encima de 35ºC.

## 7. Corrección de los datos de humedad relativa

Ahora que conocemos el efecto de la corrección sobre la simulación histórica (reducir sesgos) y sobre la señal de cambio climático (en este modelo con _eqm_, reducir la señal original), continuamos con el **objetivo de obtener proyecciones corregidas de un índice de estrés térmico**. Para ello, en primer lugar, tenemos que leer los datos de humedad relativa, de la simulación histórica y de la simulación de futuro, y corregirlos usando el método _eqm_.

In [ ]:
relhum.sim.hist <- loadGridData(dataset.sim.hist, var="hurs", years= 1986:2005, lonLim = c(-82, -68), latLim = c(1, -19))
relhum.sim.fut <- loadGridData(dataset.sim.fut, var="hurs", years= 2081:2100, lonLim = c(-82, -68), latLim = c(1, -19))

In [ ]:
relhum.eqm.hist <- biasCorrection(y=relhum.obs, x=relhum.sim.hist, newdata= relhum.sim.hist, precipitation = FALSE, 
                         method="eqm", extrapolation = "constant", n.quantiles=99 )  # corrección de 99 percentiles

In [ ]:
relhum.eqm.fut <- biasCorrection(y=relhum.obs, x=relhum.sim.hist, newdata= relhum.sim.fut, precipitation = FALSE, 
                         method="eqm", extrapolation = "constant", n.quantiles=99 )  # corrección de 99 percentiles

## 8. Cálculo de estrés térmico con datos del modelo ajustado

Una vez tenemos los datos del modelo de temperatura y humedad corregidos, calculamos el índice de estrés térmico, el _simplified wet bulb globe temperature (swgt)_ como hicimos para las observaciones, para la simulación histórica corregida (1986-2005) y para la simulación de futuro (2081-2100).

In [ ]:
wbgt.eqm.hist <- tmax.eqm.hist
for(i in 1:dim(tmax.eqm.hist$Data)[2]){
	for(j in 1:dim(tmax.eqm.hist$Data)[3]){
		wbgt.eqm.hist$Data[,i,j]<- swbgt(tmax.eqm.hist$Data[,i,j], relhum.eqm.hist$Data[,i,j])	
	}
}

In [ ]:
wbgt.eqm.fut <- tmax.eqm.fut
for(i in 1:dim(tmax.eqm.fut$Data)[2]){
	for(j in 1:dim(tmax.eqm.fut$Data)[3]){
		wbgt.eqm.fut$Data[,i,j]<- swbgt(tmax.eqm.fut$Data[,i,j], relhum.eqm.fut$Data[,i,j])	
	}
}

### 8.1. Cambios en el estrés térmico medio

Vamos a cuantificar el cambio en el _swbgt_ medio calculando la diferencia entre el valor promedio en el futuro y en el histórico (señal de cambio) , usando `gridArithmetics`. 

In [ ]:
ccs.wbgt <- gridArithmetics(climatology(wbgt.eqm.fut), climatology(wbgt.eqm.hist), operator="-")

In [ ]:
spatialPlot(ccs.wbgt, backdrop.theme = "countries", 
            color.theme = "YlOrRd",
            main=  "Señal de cambio en estrés térmico (2081-2100 wrt 1986-2005)")

Observamos un patrón espacial muy parecido al que se obtenía para el cambio medio en la temperatura.

### 8.2. Cambios en la superación de umbrales relevantes

Como se comentaba inicialmente, la corrección o ajuste de sesgos es especialmente importante cuando se trabaja con la superación de umbrales absolutos. Nos interesa conocer cómo evolucionará en el futuro el número de días con _swbgt_ superior a 26ºC y 28ºC, que habíamos analizado en la sección 3 con las observaciones. De nuevo usamos `indexGrid` para calcular el indicador, para las simulaciones histórica y futura corregidas.

In [ ]:
wbgt26.eqm.hist <- indexGrid(tx = wbgt.eqm.hist, time.resolution = "year", index.code = "TXth", th=26)
wbgt26.eqm.fut <- indexGrid(tx = wbgt.eqm.fut, time.resolution = "year", index.code = "TXth", th=26)
wbgt28.eqm.hist <- indexGrid(tx = wbgt.eqm.hist, time.resolution = "year", index.code = "TXth", th=28)
wbgt28.eqm.fut <- indexGrid(tx = wbgt.eqm.fut, time.resolution = "year", index.code = "TXth", th=28)

La siguiente figura representa en filas los dos niveles de estrés térmico considerados (_swbgt_ > 26ºC para personas no aclimatadas, _swbgt_>28ºC para personas aclimatadas), y en columnas los resultados en el periodo histórico (muy similar a los valores observados) y en el periodo futuro (2081-2100). En el futuro, las dos situaciones se vuelven más frecuentes y en las regiones del este del dominio se extenderán a prácticamente todo el año.

In [ ]:
options(repr.plot.width=12, repr.plot.height=10)
spatialPlot(makeMultiGrid(climatology(wbgt26.eqm.hist),climatology(wbgt26.eqm.fut), climatology(wbgt28.eqm.hist),climatology(wbgt28.eqm.fut), skip.temporal.check = TRUE),
            backdrop.theme = "countries", as.table=TRUE, layout=c(2,2), 
            names.attr=c("No aclimatados histórico (WBGT > 26)", "No aclimatados futuro (WBGT > 26)", "Aclimatados histórico (WBGT > 28)", "Aclimatados futuro (WBGT > 28)"), 
            rev.colors = TRUE, main= "Días con estrés térmico  (1986-2005 y 2081-2100)")

Analizamos la evolución temporal en el periodo futuro en Pucallpa, una localidad ya afectada por estrés térmico en la actualidad.

In [ ]:
wbgt26.fut.1grid <- subsetDimension(subsetDimension(wbgt26.eqm.fut, dimension = "lat", indices = 44), dimension = "lon", indices = 31)
wbgt28.fut.1grid <- subsetDimension(subsetDimension(wbgt28.eqm.fut, dimension = "lat", indices = 44), dimension = "lon", indices = 31)

In [ ]:
temporalPlot(wbgt26.fut.1grid,wbgt28.fut.1grid, cols=c("dodgerblue","coral1"), lwd = 3, 
	xyplot.custom = list(main = "Evolución temporal del estrés térmico en Pucallpa",  ylab= "Nº de días", scales=list(x=list(cex=1), y=list(cex=1)), 
                         key = list(space = "right", points = list(pch= 15, col = c("dodgerblue","coral1"), cex = 1.8), text = list(c("WBGT>26 FUT","WBGT>28 FUT"), cex = 1.5))))

## 9. Consideraciones finales

Se debe tener en cuenta, en cualquier estudio de cambio climático, que se deben considerar distintas simulaciones de modelos para poder dar una estimación robusta y fiable de los cambios proyectados. Es decir, se trabaja habitualmente con rangos de incertidumbre, que cubren los resultados de distintos modelos, no un valor único. Asimismo, se deben considerar distintos métodos de ajuste de sesgos, ya que cada uno de ellos puede tener un efecto distinto en la señal de cambio original del modelo.

## 10. Ejercicios propuestos

* Realiza el análisis para otro índice de estrés térmico que también dependa de temperatura máxima y humedad relativa media diarias. ¿Obtienes las mismas conclusiones?
* Realiza el análisis para otro método de corrección de sesgos. ¿Modifica de la misma manera la señal de cambio del índice?
* En esta práctica se ha analizado una sola simulación (modelo regional REMO2015 anidado al modelo global MPI-ESM-MR). Realiza el análisis para otros modelos regionales. Observarás que los sesgos de los modelos sin corregir (_raw_) dependen enormemente del modelo.

  Prueba con otro modelo regional anidado al mismo modelo global:
  * dataset.sim.hist <-"../shared/data/ncml/ESGF/interp025/CORDEX/output/SAM-22/ICTP/**MPI-M-MPI-ESM-MR**/historical/r1i1p1/**RegCM4-7**/v0/day/CORDEX_output_SAM-22_ICTP_MPI-M-MPI-ESM-MR_historical_r1i1p1_RegCM4-7_v0_day.ncml"
  * dataset.sim.fut <-"../shared/data/ncml/ESGF/interp025/CORDEX/output/SAM-22/ICTP/**MPI-M-MPI-ESM-MR**/rcp85/r1i1p1/**RegCM4-7**/v0/day/CORDEX_output_SAM-22_ICTP_MPI-M-MPI-ESM-MR_rcp85_r1i1p1_RegCM4-7_v0_day.ncml"
    

  Y prueba con el mismo modelo regional anidado a otro modelo global:
  * dataset.sim.hist <-"../shared/data/ncml/ESGF/interp025/CORDEX/output/SAM-22/GERICS/**NCC-NorESM1-M**/historical/r1i1p1/**REMO2015**/v1/day/CORDEX_output_SAM-22_GERICS_NCC-NorESM1-M_historical_r1i1p1_REMO2015_v1_day.ncml"
  * dataset.sim.fut <-"../shared/data/ncml/ESGF/interp025/CORDEX/output/SAM-22/GERICS/**NCC-NorESM1-M**/rcp85/r1i1p1/**REMO2015**/v1/day/CORDEX_output_SAM-22_GERICS_NCC-NorESM1-M_rcp85_r1i1p1_REMO2015_v1_day.ncml"